In [11]:
import sys
import os
import transformers
import json
import torch
import numpy as np
from torch.utils.data import DataLoader , Dataset

In [2]:
sys.path.append("../")

In [3]:
#loading in the JSON list of dictionaries
data = json.load(open("../data/geneset_dict.json"))

In [6]:
for item in data:
    print(item)
    break

{'Symbol': 'A1BG', 'Gene_ID': 'ENSG00000121410', 'Name': 'alpha-1-B glycoprotein', 'Aliases': ['A1B', 'ABG', 'GAB', 'HYST2477'], 'Other names': ['HEL-S-163pA', 'alpha-1B-glycoprotein', 'epididymis secretory sperm binding protein Li 163pA'], 'Description': 'The protein encoded by this gene is a plasma glycoprotein of unknown function. The protein shows sequence similarity to the variable regions of some immunoglobulin supergene family member proteins. [provided by RefSeq, Jul 2008].'}


In [15]:
sampler = DataLoader(data, batch_size=1, shuffle=True)
sample = next(iter(sampler))

In [16]:
print(sample)

{'Symbol': ['RNU6-489P'], 'Gene_ID': ['ENSG00000277341'], 'Name': ['RNA, U6 small nuclear 489, pseudogene'], 'Aliases': ['N/A'], 'Other names': ['N/A'], 'Description': ['N/A']}


In [18]:
query = """
Format this data into a question and answer pair. Use the information from some keys to make the question, and the information from remaining keys to make the answer. Format your question to resemble a user query. Ignore any objects with ['N/A'] as their value.
Provide the pair in this format: <question>question here</question><answer>answer here</answer>

"""

In [19]:
print(query + str(sample))


Format this data into a question and answer pair. Use the information from some keys to make the question, and the information from remaining keys to make the answer. Format your question to resemble a user query. Ignore any objects with ['N/A'] as their value.
Provide the pair in this format: <question>question here</question><answer>answer here</answer>

{'Symbol': ['RNU6-489P'], 'Gene_ID': ['ENSG00000277341'], 'Name': ['RNA, U6 small nuclear 489, pseudogene'], 'Aliases': ['N/A'], 'Other names': ['N/A'], 'Description': ['N/A']}


In [21]:
#load in llama model
from transformers import pipeline

model_id = "meta-llama/Llama-3.1-8B-Instruct"

pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
messages = [
    {"role": "system", "content": query},
    {"role": "user", "content": str(sample)},
]
outputs = pipe(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]